In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.8 MB/s 
     |████████████████████████████████| 6.6 MB 70.1 MB/s 
     |████████████████████████████████| 120 kB 75.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 15.4 MB/s 
     |████████████████████████████████| 212 kB 92.2 MB/s 
     |████████████████████████████████| 115 kB 87.3 MB/s 
     |████████████████████████████████| 127 kB 77.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


## module import


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import torch

import re
import os

import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence, Dataset

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split as tts

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

import matplotlib.pyplot as plt

print('torch version:', torch.__version__)
print('tf version:', tf.__version__)

torch version: 1.12.1+cu113
tf version: 2.8.2


In [ ]:
# random seed fix
import random

random.seed(2022)
torch.manual_seed(2022)
np.random.seed(2022)

## data load & EDA

In [ ]:
data_path = '/content/mydrive/MyDrive/AIFFELTHON/nlp_data/data_processing_re(82681).xlsx'
data = pd.read_excel(data_path)

In [ ]:
data.head()

,content,emotion,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,긴장,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,긴장,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,기쁨,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,평화,2


In [ ]:
data = data.iloc[:,:2]
data.head()

,content,emotion
0,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,긴장
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,긴장
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,기쁨
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,평화


In [ ]:
data.isna().sum()

content    1
emotion    0
dtype: int64

In [ ]:
data = data.dropna()
data.reset_index(drop=True)

,content,emotion
0,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,긴장
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,긴장
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,기쁨
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,평화
...,...,...
82675,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,분노
82676,재미가 없으니 망하지,분노
82677,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,분노
82678,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,분노


In [ ]:
data['emotion'].unique()

array(['기쁨', '긴장', '평화', '슬픔', '분노', '중립'], dtype=object)

In [ ]:
data.loc[(data['emotion'] == "기쁨"), 'emotion'] = 0  #기쁨 => 0
data.loc[(data['emotion'] == "긴장"), 'emotion'] = 1  #긴장 => 1
data.loc[(data['emotion'] == "평화"), 'emotion'] = 2  #평화 => 2
data.loc[(data['emotion'] == "슬픔"), 'emotion'] = 3  #슬픔 => 3
data.loc[(data['emotion'] == "분노"), 'emotion'] = 4  #분노 => 4
data.loc[(data['emotion'] == "중립"), 'emotion'] = 5  #중립 => 5

In [ ]:
data.head()

,content,emotion
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,2


In [ ]:
data_pro = data.copy()
data_pro = data_pro.rename(columns={'emotion':'label'})
data_pro.head()

,content,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,2


## data split

In [ ]:
train_data, val_data = tts(data_pro, test_size=0.2)

In [ ]:
print(len(train_data))
print(len(val_data))

66144
16536


## input data transform

In [ ]:
# load model, tokenizer
num_labels = 6
model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large', num_labels=num_labels)

# freeze layers
for param in model.roberta.parameters():
    param.requires_grad = False

tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classif

In [ ]:
# tokenizing test
tokenizer(train_data['content'][0])

{'input_ids': [0, 4582, 2116, 7310, 6831, 2205, 2318, 859, 2051, 2112, 3944, 1327, 2052, 717, 18, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# preprocessing function
def preprocess_function(data):
    return tokenizer(
        # tokenizing
        data['content'],

        max_length=512,

        # 최대 길이보다 긴 시퀀스는 최대 길이에 맞춰 자름
        truncation=True,

        # tokenizer가 token_type_ids를 return하지 않게 함
        # roberta는 필요없기 때문
        return_token_type_ids=False,
    )

In [ ]:
# dataset transform
df_train = pd.DataFrame({'content':train_data['content'], 'label':train_data['label']})
dataset_train = Dataset.from_pandas(df_train)

df_val = pd.DataFrame({'content':val_data['content'], 'label':val_data['label']})
dataset_val = Dataset.from_pandas(df_val)

In [ ]:
tokenized_train_datasets = dataset_train.map(preprocess_function, batched=True)
tokenized_val_datasets = dataset_val.map(preprocess_function, batched=True)

  0%|          | 0/67 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

In [ ]:
tokenized_train_datasets

Dataset({
    features: ['content', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 66144
})

In [ ]:
tokenized_val_datasets

Dataset({
    features: ['content', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 16536
})

## eval metric 정의

In [ ]:
'''
multi sentiment analysis는 기본적으로 문장 분류 문제
KLUE task를 학습한 klue-roberta 중에서 TC(topic classification)와 유사(완전히 같지는 않음)
TC는 평가 지표로 macro f1 score를 사용

but klue에서는 다중감성분류를 학습하지 않았고,
glue에서는 sst2라는 긍부정 분류 task가 있는데 accuracy를 사용

but 우리는 데이터셋이 다소 imbalance하기 때문에 accuracy를 채택하면 오차가 발생할 수 있음

따라서 더 data imbalance에 robust한 metric를 만들기 위해
sklearn에서 제공하는 f1 score metric을 활용
'''

def eval_metric(pred, real):
    f1 = {'weighted_f1':f1_score(real, pred, average='weighted')}
    return f1

In [ ]:
# metric test
fake_preds = np.random.randint(0, 6, size=(64,))
fake_labels = np.random.randint(0, 6, size=(64,))
fake_preds, fake_labels

(array([4, 1, 0, 5, 3, 4, 4, 4, 2, 4, 1, 0, 4, 0, 1, 3, 5, 3, 0, 0, 3, 0,
        4, 0, 5, 4, 4, 3, 5, 4, 4, 5, 2, 4, 0, 2, 5, 0, 2, 2, 2, 5, 2, 3,
        4, 5, 1, 3, 3, 1, 4, 0, 1, 0, 0, 2, 2, 5, 4, 3, 2, 2, 0, 1]),
 array([4, 5, 1, 4, 1, 3, 3, 4, 1, 5, 1, 0, 5, 1, 1, 0, 2, 4, 1, 5, 4, 1,
        4, 5, 3, 0, 4, 4, 4, 4, 1, 2, 1, 0, 5, 0, 0, 5, 0, 4, 2, 1, 5, 1,
        1, 4, 1, 1, 5, 4, 0, 1, 2, 2, 4, 2, 1, 4, 4, 0, 1, 5, 3, 5]))

In [ ]:
eval_metric(fake_preds, fake_labels)

{'weighted_f1': 0.1991171079868898}

In [ ]:
# eval metric
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    result = eval_metric(predictions, labels)
    return result

## train arg

In [ ]:
# metric_name은 eval_metric에서 리턴받는 dict 형태의 키 이름
metric_name = "weighted_f1"

# batch size 지정
batch_size = 8

args = TrainingArguments(
    "baseline_test",
    # evaluation_strategy="epoch",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: content, __index_level_0__. If content, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 66144
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 41340


Step,Training Loss,Validation Loss,Weighted F1
500,1.592900,1.541486,0.337133
1000,1.526500,1.499867,0.308674
1500,1.481900,1.463648,0.354714
2000,1.451100,1.436126,0.418062
2500,1.428100,1.397606,0.445662
3000,1.400400,1.372003,0.432191
3500,1.358700,1.349499,0.446191
4000,1.339700,1.333672,0.456412
4500,1.319000,1.295795,0.480290
5000,1.311700,1.275536,0.480969


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: content, __index_level_0__. If content, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 16536
  Batch size = 8
Saving model checkpoint to baseline_test/checkpoint-500
Configuration saved in baseline_test/checkpoint-500/config.json
Model weights saved in baseline_test/checkpoint-500/pytorch_model.bin
tokenizer config file saved in baseline_test/checkpoint-500/tokenizer_config.json
Special tokens file saved in baseline_test/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: content, __index_level_0__. If content, __index_level_0__ are not expected by `RobertaForSequenceClassification.forwar

TrainOutput(global_step=16000, training_loss=1.2662486038208007, metrics={'train_runtime': 2495.5854, 'train_samples_per_second': 132.522, 'train_steps_per_second': 16.565, 'total_flos': 6909214084161312.0, 'train_loss': 1.2662486038208007, 'epoch': 1.94})